In [ ]:
# %pip install -q -U google-genai
# %pip install python-dotenv

import os
import requests
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pathlib import Path

load_dotenv()

In [ ]:
payload = {
    "grant_type": "password",
    'username': os.getenv("TRANSKRIBUS_USER"),
    'password': os.getenv("TRANSKRIBUS_PASS"),
    'client_id': 'processing-api-client',
}

url = "https://account.readcoop.eu/auth/realms/readcoop/protocol/openid-connect/token"
response = requests.post(url, data=payload)
response.raise_for_status() 
token = response.json().get('access_token')

In [ ]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.

client = genai.Client()

folder_path = Path("Liber Testamentorum")

image_paths = list(folder_path.glob("*.jpg"))
with open(image_paths[0], "rb") as image_file:
    image_data = image_file.read()
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=[
            types.Part.from_bytes(
                data=image_data,
                mime_type='image/jpeg'
            ),
            "Please scan this image of a Polish 19th century document for all names. Return only the names in a comma separated list, do NOT return any other text."
            ]
    )
    print(response.text)

""""
for image_path in folder_path.glob("*.jpg"):
    print(f"Processing {image_path}...")
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
        response = client.models.generate_content(
            model="gemini-3-flash-preview",
            contents=["Please scan this image of a Polish 19th century document for all names. Return only the names found, separated by commas.", image_data],
            content_type="image/jpeg",
        )
        print(response.text)
"""
